### 환경 설정

In [ ]:
import os
import json
import time

import httpx
from dotenv import load_dotenv
load_dotenv()

In [ ]:
network = "ethereum"
collection_id = "0xed5af388653567af2f388e6224dc7c4b3241c544"

In [ ]:
def write_json(data, filepath):
    with open(filepath, 'w') as file:
        json.dump(data, file, ensure_ascii=False)

In [ ]:
url_collection = "https://api.reservoir.tools/collections/v7"
url_tokens = "https://api.reservoir.tools/tokens/v7"
url_transaction = "https://api.reservoir.tools/collections/activity/v6"
url_attributes = "https://api.reservoir.tools/collections/{collection}/attributes/all/v4"


headers = {
    "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    "accept": "*/*"
}

In [ ]:
path_output = "./2nd"
path_tokens = os.path.join(path_output, "tokens")
path_transactions = os.path.join(path_output, "transactions")
path_collections = os.path.join(path_output, "collections")
path_attributes = os.path.join(path_output, "attributes")

output_collection_name = "{network}_{collection_id}_collection.json"
output_tokens_name = "{network}_{collection_id}_{idx:06d}_tokens.json"
output_transactions_name = "{network}_{collection_id}_{idx:06d}_transactions.json"
output_attributes_name = "{network}_{collection_id}_attributes.json"

os.makedirs(path_output, exist_ok=True)
os.makedirs(path_tokens, exist_ok=True)
os.makedirs(path_transactions, exist_ok=True)
os.makedirs(path_collections, exist_ok=True)
os.makedirs(path_attributes, exist_ok=True)

### Collections

In [ ]:
collection = httpx.get(url_collection, params={"id": collection_id}, headers=headers).json()
write_json(collection, os.path.join(path_collections, output_collection_name.format(network=network, collection_id=collection_id)))
collection

### Tokens

In [ ]:
idx = 0
continuation = ""
fault_count = 0

while True:
    params = {
        "collection": collection_id,
        "limit": 1000,
        "sortBy": "updatedAt",
        "includeAttributes": True,
    }
    
    if continuation:
        params["continuation"] = continuation
        
    print(params)
    
    resp = httpx.get(url_tokens, params=params, headers=headers, timeout=30).json()
    
    # 500번 에러일 떄 단순 요청 미스라 판단하고 다시 요청
    if resp.status_code == 500:
        if fault_count > 3:
            time.sleep(10)
            fault_count += 1
            continue
        elif fault_count > 6:
            print(f"Too many faults: {idx}, {network}, {collection_id}, {continuation}")
            break
        else:
            fault_count += 1
            continue
    # 429 에러일 떄 10초 대기 후 다시 요청
    elif resp.status_code == 429:
        time.sleep(10)
        continue
    else:
        resp = resp.json()
        fault_count = 0
    
    
    write_json(resp, os.path.join(path_tokens, output_tokens_name.format(network=network, collection_id=collection_id, idx=idx)))
    
    print(f"idx: {idx}, Downloaded {len(resp['tokens'])} tokens, continuation: {resp['continuation']}")
    if len(resp["tokens"]) < 1000 or resp["continuation"] is None:
        break
    
    idx += 1
    continuation = resp["continuation"]

### Transactions

In [ ]:
idx = 0
continuation = ""
fault_count = 0

while True:
    params = {
        "collection": collection_id,
        "limit": 50,
        "types": ["sale", "transfer", "mint"]
    }
    
    if continuation:
        params["continuation"] = continuation
        
    print(params)
    
    resp = httpx.get(url_transaction, params=params, headers=headers, timeout=30)
    
    # 500번 에러일 떄 단순 요청 미스라 판단하고 다시 요청
    if resp.status_code == 500:
        if fault_count > 3:
            time.sleep(10)
            fault_count += 1
            continue
        elif fault_count > 6:
            print(f"Too many faults: {idx}, {network}, {collection_id}, {continuation}")
            break
        else:
            fault_count += 1
            continue
    # 429 에러일 떄 10초 대기 후 다시 요청
    elif resp.status_code == 429:
        time.sleep(10)
        continue
    else:
        resp = resp.json()
        fault_count = 0
    
    write_json(resp, os.path.join(path_transactions, output_transactions_name.format(network=network, collection_id=collection_id, idx=idx)))
    
    print(f"idx: {idx}, Downloaded {len(resp['activities'])} activities, continuation: {resp['continuation']}")
    if len(resp["activities"]) < 50 or resp["continuation"] is None:
        break
    
    idx += 1
    continuation = resp["continuation"]

### Attributes

In [ ]:
resp = httpx.get(url_attributes.format(collection=collection_id), headers=headers).json()
resp["network"] = network
resp["id"] = collection_id
write_json(resp, os.path.join(path_attributes, output_attributes_name.format(network=network, collection_id=collection_id)))
resp